<a href="https://colab.research.google.com/github/noorelhaj98-ship-it/pytorch-tutorial/blob/main/traffic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="snA4vsNSH6S3ia5q9egX")
project = rf.workspace("traffic-tmfdb").project("vechiles-hq6la-r7fco")
version = project.version(2)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 111.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Vechiles-2 in yolov8:: 100%|██████████| 16102/16102 [00:03<00:00, 5021.75it/s]


In [2]:
import os

images_dir = "/content/Vechiles-2/train/images"
labels_dir = "/content/Vechiles-2/train/labels"

image_exts = (".jpg", ".jpeg", ".png")

images = sorted([
    os.path.splitext(f)[0]
    for f in os.listdir(images_dir)
    if f.lower().endswith(image_exts)
])

labels = sorted([
    os.path.splitext(f)[0]
    for f in os.listdir(labels_dir)
    if f.lower().endswith(".txt")
])

print("Total images:", len(images))
print("Total labels:", len(labels))

image_set = set(images)
label_set = set(labels)

images_without_labels = image_set - label_set
labels_without_images = label_set - image_set

if not images_without_labels and not labels_without_images:
    print("\n✅ RESULT: Every image has a matching label and vice versa")
else:
    print("\n❌ RESULT: Mismatches found")

    if images_without_labels:
        print("\nImages without labels:")
        for f in sorted(images_without_labels)[:10]:
            print(" ", f)

    if labels_without_images:
        print("\nLabels without images:")
        for f in sorted(labels_without_images)[:10]:
            print(" ", f)


Total images: 8048
Total labels: 8048

✅ RESULT: Every image has a matching label and vice versa


## ✅ Preprocessing + preparation (clean → resize/normalize → augment → YOLO labels)

This section:
- Cleans YOLO labels (removes invalid boxes, clips to bounds, removes duplicates)
- Resizes images to a fixed `imgsz` (letterbox) and updates labels
- Applies data augmentation (train only)
- Writes a new YOLOv8-ready dataset folder and `data.yaml`

In [ ]:
# Install deps (Colab)
!pip -q install albumentations opencv-python pyyaml

import os, glob, math, shutil
import cv2
import numpy as np
import albumentations as A
import yaml
from pathlib import Path

# ---- CONFIG ----
DATA_ROOT = "/content/Vechiles-2"      # change if your folder name differs
OUT_ROOT  = "/content/Vechiles-2-prep" # output dataset (YOLOv8 format)
IMGSZ     = 640                        # model input size (e.g., 640 / 832 / 1024)
AUG_PER_IMAGE = 2                      # augmentations per train image
MIN_BOX_AREA_PX = 16                   # drop very tiny boxes (area in pixels)
DUP_IOU = 0.98                         # IoU threshold to treat boxes as duplicates (same class)
# ----------------

SPLITS = []
for s in ["train", "val", "valid", "test"]:
    if os.path.isdir(os.path.join(DATA_ROOT, s, "images")) and os.path.isdir(os.path.join(DATA_ROOT, s, "labels")):
        SPLITS.append(s)

print("Detected splits:", SPLITS)
assert len(SPLITS) > 0, "No splits found. Expected folders like DATA_ROOT/train/images and DATA_ROOT/train/labels"

In [ ]:
def yolo_to_xyxy_px(yolo_box, w, h):
    # yolo_box: (xc,yc,bw,bh) normalized -> xyxy in pixels
    xc, yc, bw, bh = yolo_box
    x1 = (xc - bw/2) * w
    y1 = (yc - bh/2) * h
    x2 = (xc + bw/2) * w
    y2 = (yc + bh/2) * h
    return np.array([x1, y1, x2, y2], dtype=float)

def xyxy_px_to_yolo(xyxy, w, h):
    x1, y1, x2, y2 = xyxy
    bw = max(0.0, x2 - x1)
    bh = max(0.0, y2 - y1)
    xc = x1 + bw/2
    yc = y1 + bh/2
    return np.array([xc / w, yc / h, bw / w, bh / h], dtype=float)

def clip_xyxy(xyxy, w, h):
    x1, y1, x2, y2 = xyxy
    x1 = max(0.0, min(x1, w - 1.0))
    y1 = max(0.0, min(y1, h - 1.0))
    x2 = max(0.0, min(x2, w - 1.0))
    y2 = max(0.0, min(y2, h - 1.0))
    return np.array([x1, y1, x2, y2], dtype=float)

def area_xyxy(xyxy):
    x1, y1, x2, y2 = xyxy
    return max(0.0, x2 - x1) * max(0.0, y2 - y1)

def iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    ix1, iy1 = max(ax1, bx1), max(ay1, by1)
    ix2, iy2 = min(ax2, bx2), min(ay2, by2)
    inter = max(0.0, ix2 - ix1) * max(0.0, iy2 - iy1)
    ua = area_xyxy(a) + area_xyxy(b) - inter
    return inter / ua if ua > 0 else 0.0

def remove_duplicates_xyxy(boxes_xyxy, cls_ids, iou_thresh=0.98):
    # keep larger boxes first
    order = sorted(range(len(boxes_xyxy)), key=lambda i: area_xyxy(boxes_xyxy[i]), reverse=True)
    keep = []
    used = [False]*len(order)
    for ii, i in enumerate(order):
        if used[ii]:
            continue
        keep.append(i)
        for jj, j in enumerate(order):
            if used[jj] or jj == ii:
                continue
            if cls_ids[j] != cls_ids[i]:
                continue
            if iou_xyxy(boxes_xyxy[i], boxes_xyxy[j]) >= iou_thresh:
                used[jj] = True
    keep_boxes = [boxes_xyxy[i] for i in keep]
    keep_cls = [cls_ids[i] for i in keep]
    return keep_boxes, keep_cls

def read_yolo_label(path):
    boxes, cls_ids = [], []
    if not os.path.exists(path):
        return boxes, cls_ids
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) != 5:
                continue
            c = int(float(parts[0]))
            xc, yc, bw, bh = map(float, parts[1:])
            boxes.append([xc, yc, bw, bh])
            cls_ids.append(c)
    return boxes, cls_ids

def write_yolo_label(path, boxes, cls_ids):
    with open(path, "w", encoding="utf-8") as f:
        for b, c in zip(boxes, cls_ids):
            xc, yc, bw, bh = b
            f.write(f"{c} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n")

def letterbox(image, new_size=640, color=(114,114,114)):
    # Resize with padding to (new_size,new_size). Returns image, scale, pad (dw,dh).
    h, w = image.shape[:2]
    r = min(new_size / h, new_size / w)
    new_unpad = (int(round(w * r)), int(round(h * r)))
    resized = cv2.resize(image, new_unpad, interpolation=cv2.INTER_LINEAR)

    dw = new_size - new_unpad[0]
    dh = new_size - new_unpad[1]
    dw /= 2
    dh /= 2

    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))

    padded = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    return padded, r, (left, top)

def clean_yolo_boxes(yolo_boxes, cls_ids, w, h, min_area_px=16, dup_iou=0.98):
    # Clip boxes, remove invalid/small, remove duplicates (same class).
    cleaned_boxes, cleaned_cls = [], []
    for b, c in zip(yolo_boxes, cls_ids):
        if any(math.isnan(float(x)) for x in b):
            continue
        xc, yc, bw, bh = map(float, b)
        if bw <= 0 or bh <= 0:
            continue

        xyxy = yolo_to_xyxy_px((xc, yc, bw, bh), w, h)
        xyxy = clip_xyxy(xyxy, w, h)
        if area_xyxy(xyxy) < min_area_px:
            continue

        yolo = xyxy_px_to_yolo(xyxy, w, h)
        yolo = np.clip(yolo, 0.0, 1.0)
        cleaned_boxes.append(yolo.tolist())
        cleaned_cls.append(int(c))

    if len(cleaned_boxes) > 1:
        boxes_xyxy = [yolo_to_xyxy_px(b, w, h) for b in cleaned_boxes]
        boxes_xyxy, cleaned_cls = remove_duplicates_xyxy(boxes_xyxy, cleaned_cls, iou_thresh=dup_iou)
        cleaned_boxes = [np.clip(xyxy_px_to_yolo(b, w, h), 0.0, 1.0).tolist() for b in boxes_xyxy]

    return cleaned_boxes, cleaned_cls

In [ ]:
# Augmentations (train only). Albumentations handles bbox updates correctly.
augmenter = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.Affine(scale=(0.85, 1.15), translate_percent=(0.0, 0.05), rotate=(-12, 12), p=0.8),
        A.RandomResizedCrop(height=IMGSZ, width=IMGSZ, scale=(0.75, 1.0), ratio=(0.9, 1.1), p=0.6),
    ],
    bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"], min_visibility=0.2),
)

In [ ]:
from tqdm import tqdm

# Create output folders
for split in SPLITS:
    Path(OUT_ROOT, "images", split).mkdir(parents=True, exist_ok=True)
    Path(OUT_ROOT, "labels", split).mkdir(parents=True, exist_ok=True)

image_exts = (".jpg", ".jpeg", ".png")

stats = {s: {"images":0, "kept_boxes":0, "dropped_boxes":0} for s in SPLITS}

for split in SPLITS:
    in_img_dir = os.path.join(DATA_ROOT, split, "images")
    in_lbl_dir = os.path.join(DATA_ROOT, split, "labels")
    out_img_dir = os.path.join(OUT_ROOT, "images", split)
    out_lbl_dir = os.path.join(OUT_ROOT, "labels", split)

    img_files = []
    for ext in image_exts:
        img_files.extend(glob.glob(os.path.join(in_img_dir, f"*{ext}")))
    img_files = sorted(img_files)

    for img_path in tqdm(img_files, desc=f"Preprocess {split}"):
        base = os.path.splitext(os.path.basename(img_path))[0]
        lbl_path = os.path.join(in_lbl_dir, base + ".txt")

        img = cv2.imread(img_path)
        if img is None:
            continue
        h, w = img.shape[:2]

        yolo_boxes, cls_ids = read_yolo_label(lbl_path)
        before = len(yolo_boxes)

        # clean labels at original resolution
        yolo_boxes, cls_ids = clean_yolo_boxes(yolo_boxes, cls_ids, w, h, min_area_px=MIN_BOX_AREA_PX, dup_iou=DUP_IOU)
        after_clean = len(yolo_boxes)

        # resize/normalize image to IMGSZ with letterbox
        img_lb, r, (padx, pady) = letterbox(img, new_size=IMGSZ)

        # update boxes to new letterboxed coords
        new_boxes, new_cls = [], []
        for b, c in zip(yolo_boxes, cls_ids):
            xyxy = yolo_to_xyxy_px(b, w, h)  # original px
            xyxy *= r
            xyxy[[0,2]] += padx
            xyxy[[1,3]] += pady
            xyxy = clip_xyxy(xyxy, IMGSZ, IMGSZ)
            if area_xyxy(xyxy) < MIN_BOX_AREA_PX:
                continue
            new_boxes.append(np.clip(xyxy_px_to_yolo(xyxy, IMGSZ, IMGSZ), 0.0, 1.0).tolist())
            new_cls.append(int(c))

        # write base resized sample
        out_img_path = os.path.join(out_img_dir, base + ".jpg")
        out_lbl_path = os.path.join(out_lbl_dir, base + ".txt")
        cv2.imwrite(out_img_path, img_lb)
        write_yolo_label(out_lbl_path, new_boxes, new_cls)

        stats[split]["images"] += 1
        stats[split]["kept_boxes"] += len(new_boxes)
        stats[split]["dropped_boxes"] += (before - after_clean) + max(0, after_clean - len(new_boxes))

        # augment only train split
        if split == "train" and AUG_PER_IMAGE > 0 and len(new_boxes) > 0:
            for k in range(AUG_PER_IMAGE):
                aug = augmenter(image=img_lb, bboxes=new_boxes, class_labels=new_cls)
                aug_img = aug["image"]
                aug_boxes = list(aug["bboxes"])
                aug_cls = list(aug["class_labels"])

                # clean after augmentation (tiny boxes after crop)
                filt_boxes, filt_cls = [], []
                for bb, cc in zip(aug_boxes, aug_cls):
                    xyxy = yolo_to_xyxy_px(bb, IMGSZ, IMGSZ)
                    xyxy = clip_xyxy(xyxy, IMGSZ, IMGSZ)
                    if area_xyxy(xyxy) >= MIN_BOX_AREA_PX:
                        filt_boxes.append(np.clip(xyxy_px_to_yolo(xyxy, IMGSZ, IMGSZ), 0.0, 1.0).tolist())
                        filt_cls.append(int(cc))

                aug_name = f"{base}_aug{k}"
                cv2.imwrite(os.path.join(out_img_dir, aug_name + ".jpg"), aug_img)
                write_yolo_label(os.path.join(out_lbl_dir, aug_name + ".txt"), filt_boxes, filt_cls)

print("✅ Done preprocessing.")
stats

In [ ]:
# Create data.yaml for YOLOv8
# If you know the class names, set NAMES = ["car", "bus", ...] below.

NAMES = None  # e.g. ["car","bus","truck","motorcycle","person"]

if NAMES is None:
    all_cls = set()
    for split in SPLITS:
        for p in glob.glob(os.path.join(OUT_ROOT, "labels", split, "*.txt")):
            with open(p, "r", encoding="utf-8") as f:
                for line in f:
                    line=line.strip()
                    if not line:
                        continue
                    all_cls.add(int(float(line.split()[0])))
    n = (max(all_cls)+1) if all_cls else 1
    NAMES = [f"class_{i}" for i in range(n)]

data_yaml = {
    "path": OUT_ROOT,
    "train": "images/train" if "train" in SPLITS else f"images/{SPLITS[0]}",
    "val": "images/val" if "val" in SPLITS else ("images/valid" if "valid" in SPLITS else f"images/{SPLITS[0]}"),
    "names": {i:name for i, name in enumerate(NAMES)}
}

yaml_path = os.path.join(OUT_ROOT, "data.yaml")
with open(yaml_path, "w", encoding="utf-8") as f:
    yaml.safe_dump(data_yaml, f, sort_keys=False)

print("Wrote:", yaml_path)
print(data_yaml)

In [ ]:
# Train YOLOv8 on the preprocessed dataset (optional)
# !pip -q install ultralytics
# !yolo detect train model=yolov8n.pt data=/content/Vechiles-2-prep/data.yaml imgsz=640 epochs=50

In [3]:
import os
from PIL import Image


In [4]:
images_dir = "/content/Vechiles-2/train/images"  # change if needed


In [5]:
converted = 0
skipped = 0

for filename in os.listdir(images_dir):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        img_path = os.path.join(images_dir, filename)

        try:
            with Image.open(img_path) as img:
                # Check image mode
                if img.mode != "RGB":
                    img = img.convert("RGB")
                    img.save(img_path)
                    converted += 1
                else:
                    skipped += 1

        except Exception as e:
            print(f"❌ Error processing {filename}: {e}")

print(f"✅ Converted to RGB: {converted}")
print(f"⏭️ Already RGB (skipped): {skipped}")


✅ Converted to RGB: 0
⏭️ Already RGB (skipped): 8048


In [6]:
import os
from PIL import Image, ImageOps


In [7]:
fixed = 0
skipped = 0

for filename in os.listdir(images_dir):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        img_path = os.path.join(images_dir, filename)

        try:
            with Image.open(img_path) as img:
                # Apply EXIF-based auto orientation
                img_oriented = ImageOps.exif_transpose(img)

                # Save back only if something changed
                if img_oriented != img:
                    img_oriented.save(img_path)
                    fixed += 1
                else:
                    skipped += 1

        except Exception as e:
            print(f"❌ Error processing {filename}: {e}")

print(f"🔄 Auto-oriented images: {fixed}")
print(f"⏭️ No orientation change needed: {skipped}")


🔄 Auto-oriented images: 8048
⏭️ No orientation change needed: 0


In [8]:
import os
import cv2
import numpy as np
from collections import Counter


In [9]:
base_path = "/content/Vechiles-2/train"
images_dir = os.path.join(base_path, "images")
labels_dir = os.path.join(base_path, "labels")


In [10]:
image_sizes = []

for img_name in os.listdir(images_dir):
    if img_name.lower().endswith((".jpg", ".png", ".jpeg")):
        img = cv2.imread(os.path.join(images_dir, img_name))
        h, w, _ = img.shape
        image_sizes.append((w, h))

widths, heights = zip(*image_sizes)

print("=== IMAGE SIZE ANALYSIS ===")
print(f"Total images: {len(image_sizes)}")
print(f"Min size: {min(widths)}x{min(heights)}")
print(f"Max size: {max(widths)}x{max(heights)}")
print(f"Avg size: {int(np.mean(widths))}x{int(np.mean(heights))}")


=== IMAGE SIZE ANALYSIS ===
Total images: 8048
Min size: 640x640
Max size: 640x640
Avg size: 640x640


In [11]:
empty_labels = []

for label_name in os.listdir(labels_dir):
    label_path = os.path.join(labels_dir, label_name)
    if os.path.getsize(label_path) == 0:
        empty_labels.append(label_name)

print("\n=== EMPTY LABELS ===")
print(f"Empty label files: {len(empty_labels)}")



=== EMPTY LABELS ===
Empty label files: 0


In [12]:
class_counter = Counter()

for label_name in os.listdir(labels_dir):
    with open(os.path.join(labels_dir, label_name)) as f:
        for line in f:
            class_id = int(line.split()[0])
            class_counter[class_id] += 1

print("\n=== CLASS DISTRIBUTION ===")
for cls, count in class_counter.items():
    print(f"Class {cls}: {count}")



=== CLASS DISTRIBUTION ===
Class 3: 15656
Class 1: 4237
Class 4: 21294
Class 2: 12357
Class 0: 412


In [13]:
box_areas = []
small_boxes = 0

for label_name in os.listdir(labels_dir):
    with open(os.path.join(labels_dir, label_name)) as f:
        for line in f:
            _, _, _, w, h = map(float, line.split())
            area = w * h
            box_areas.append(area)

            if area < 0.01:  # <1% of image
                small_boxes += 1

print("\n=== BOUNDING BOX ANALYSIS ===")
print(f"Total boxes: {len(box_areas)}")
print(f"Avg box area (%): {np.mean(box_areas)*100:.2f}")
print(f"Small boxes (<1%): {small_boxes}")



=== BOUNDING BOX ANALYSIS ===
Total boxes: 53956
Avg box area (%): 1.28
Small boxes (<1%): 35632


In [14]:
aspect_ratios = []

for label_name in os.listdir(labels_dir):
    with open(os.path.join(labels_dir, label_name)) as f:
        for line in f:
            _, _, _, w, h = map(float, line.split())
            if h > 0:
                aspect_ratios.append(w / h)

print("\n=== ASPECT RATIO ===")
print(f"Avg aspect ratio: {np.mean(aspect_ratios):.2f}")



=== ASPECT RATIO ===
Avg aspect ratio: 1.35
